## kitti_waymo_字典打印输出mAp
参考：pcdet/datasets/kitti/kitti_object_eval_python/eval.py

In [16]:

import io as sysio

import numba
import numpy as np

overlap_0_7 = np.array([[0.7, 0.5, 0.5, 0.7,
                            0.5, 0.7], [0.7, 0.5, 0.5, 0.7, 0.5, 0.7],
                        [0.7, 0.5, 0.5, 0.7, 0.5, 0.7]])
overlap_0_5 = np.array([[0.7, 0.5, 0.5, 0.7,
                            0.5, 0.5], [0.5, 0.25, 0.25, 0.5, 0.25, 0.5],
                        [0.5, 0.25, 0.25, 0.5, 0.25, 0.5]])

# 目的就是给不同的类别设置不同的阈值
# 每个类别输出2*2个结果，AP或AP_R40，overlap=0.7或0.5
min_overlaps = np.stack([overlap_0_7, overlap_0_5], axis=0)  # [2, 3, 5]
class_to_name = {
    0: 'Car', 			#[[0.7,0.7,0.7],[0.5, 0.5, 0.5]]
    1: 'Pedestrian',	#[[0.5,0.5,0.5],[0.5,0.25,0.25]]
    2: 'Cyclist',		#[[0.5,0.5,0.5],[0.5,0.25,0.25]]
    3: 'Van',			#[[0.7,0.7,0.7],[0.7, 0.5, 0.5]]
    4: 'Person_sitting',#[[0.5,0.5,0.5],[0.5,0.25,0.25]]
    5: 'Truck'			#[[0.7,0.7,0.7],[0.5, 0.5, 0.5]]
}
current_classes = [0, 1, 2] #只要前三类

name_to_class = {v: n for n, v in class_to_name.items()}
if not isinstance(current_classes, (list, tuple)):
    current_classes = [current_classes]
current_classes_int = []
for curcls in current_classes:
    if isinstance(curcls, str):
        current_classes_int.append(name_to_class[curcls])
    else:
        current_classes_int.append(curcls)
current_classes = current_classes_int
min_overlaps = min_overlaps[:, :, current_classes]
result = ''
# check whether alpha is valid
compute_aos = False


In [19]:
# 计算结果的函数do_eval()（后面详细说）===================================================================
# mAPbbox, mAPbev, mAP3d, mAPaos, mAPbbox_R40, mAPbev_R40, mAP3d_R40, mAPaos_R40 = do_eval(
#     gt_annos, dt_annos, current_classes, min_overlaps, compute_aos, PR_detail_dict=PR_detail_dict)

# shape(3,3,2)
mAPbbox = np.array([[[0., 0.],  [0., 0.],  [0., 0.]],
                         [[0., 0.],  [0., 0.],  [0., 0.]],
                          [[0., 0.],  [0., 0.],  [0., 0.]]])
mAPbev = np.array([[[0., 0.],  [0., 0.],  [0., 0.]],
                         [[0., 0.],  [0., 0.],  [0., 0.]],
                          [[0., 0.],  [0., 0.],  [0., 0.]]])
mAP3d = np.array([[[0., 0.],  [0., 0.],  [0., 0.]],
                         [[0., 0.],  [0., 0.],  [0., 0.]],
                          [[0., 0.],  [0., 0.],  [0., 0.]]])
mAPbbox_R40 = np.array([[[0., 0.],  [0., 0.],  [0., 0.]],
                         [[0., 0.],  [0., 0.],  [0., 0.]],
                          [[0., 0.],  [0., 0.],  [0., 0.]]])
mAPbev_R40 = np.array([[[0., 0.],  [0., 0.],  [0., 0.]],
                         [[0., 0.],  [0., 0.],  [0., 0.]],
                          [[0., 0.],  [0., 0.],  [0., 0.]]])
mAP3d_R40 = np.array([[[0., 0.],  [0., 0.],  [0., 0.]],
                         [[0., 0.],  [0., 0.],  [0., 0.]],
                          [[0., 0.],  [0., 0.],  [0., 0.]]])

In [20]:


# 打印函数
def print_str(value, *arg, sstream=None):
    if sstream is None:
        sstream = sysio.StringIO()
    sstream.truncate(0)
    sstream.seek(0)
    print(value, *arg, file=sstream)
    return sstream.getvalue()

# 打印结果
ret_dict = {}

for j, curcls in enumerate(current_classes): #每一种类
    # mAP threshold array: [num_minoverlap, metric, class]
    # mAP result: [num_class, num_diff, num_minoverlap]
    for i in range(min_overlaps.shape[0]):
        # 最终结果是reslut
        # 第一种计算方式
        # 加入f就可以起到和format函数类似的作用等效于print("Shape of tensor:{}".format(tensor.shape))
        result += print_str(
            (f"{class_to_name[curcls]} "
                "AP@{:.2f}, {:.2f}, {:.2f}:".format(*min_overlaps[i, :, j])))
        result += print_str((f"bbox AP:{mAPbbox[j, 0, i]:.4f}, " # 控制小数点位数
                                f"{mAPbbox[j, 1, i]:.4f}, "
                                f"{mAPbbox[j, 2, i]:.4f}"))  # 加入f就可以起到和format函数类似的作用等效于print("Shape of tensor:{}".format(tensor.shape))
        result += print_str((f"bev  AP:{mAPbev[j, 0, i]:.4f}, "
                                f"{mAPbev[j, 1, i]:.4f}, "
                                f"{mAPbev[j, 2, i]:.4f}"))
        result += print_str((f"3d   AP:{mAP3d[j, 0, i]:.4f}, "
                                f"{mAP3d[j, 1, i]:.4f}, "
                                f"{mAP3d[j, 2, i]:.4f}"))


        # 第二种计算方式
        result += print_str(
            (f"{class_to_name[curcls]} "
                "AP_R40@{:.2f}, {:.2f}, {:.2f}:".format(*min_overlaps[i, :, j])))
        result += print_str((f"bbox AP:{mAPbbox_R40[j, 0, i]:.4f}, "
                                f"{mAPbbox_R40[j, 1, i]:.4f}, "
                                f"{mAPbbox_R40[j, 2, i]:.4f}"))
        result += print_str((f"bev  AP:{mAPbev_R40[j, 0, i]:.4f}, "
                                f"{mAPbev_R40[j, 1, i]:.4f}, "
                                f"{mAPbev_R40[j, 2, i]:.4f}"))
        result += print_str((f"3d   AP:{mAP3d_R40[j, 0, i]:.4f}, "
                                f"{mAP3d_R40[j, 1, i]:.4f}, "
                                f"{mAP3d_R40[j, 2, i]:.4f}"))

        if i == 0:
            # 一种计算方式
            # ret_dict['%s_3d/easy' % class_to_name[curcls]] = mAP3d[j, 0, 0]
            # ret_dict['%s_3d/moderate' % class_to_name[curcls]] = mAP3d[j, 1, 0]
            # ret_dict['%s_3d/hard' % class_to_name[curcls]] = mAP3d[j, 2, 0]
            # ret_dict['%s_bev/easy' % class_to_name[curcls]] = mAPbev[j, 0, 0]
            # ret_dict['%s_bev/moderate' % class_to_name[curcls]] = mAPbev[j, 1, 0]
            # ret_dict['%s_bev/hard' % class_to_name[curcls]] = mAPbev[j, 2, 0]
            # ret_dict['%s_image/easy' % class_to_name[curcls]] = mAPbbox[j, 0, 0]
            # ret_dict['%s_image/moderate' % class_to_name[curcls]] = mAPbbox[j, 1, 0]
            # ret_dict['%s_image/hard' % class_to_name[curcls]] = mAPbbox[j, 2, 0]

            # 另一种计算方式
            ret_dict['%s_3d/easy_R40' % class_to_name[curcls]] = mAP3d_R40[j, 0, 0]
            ret_dict['%s_3d/moderate_R40' % class_to_name[curcls]] = mAP3d_R40[j, 1, 0]
            ret_dict['%s_3d/hard_R40' % class_to_name[curcls]] = mAP3d_R40[j, 2, 0]
            ret_dict['%s_bev/easy_R40' % class_to_name[curcls]] = mAPbev_R40[j, 0, 0]
            ret_dict['%s_bev/moderate_R40' % class_to_name[curcls]] = mAPbev_R40[j, 1, 0]
            ret_dict['%s_bev/hard_R40' % class_to_name[curcls]] = mAPbev_R40[j, 2, 0]
            ret_dict['%s_image/easy_R40' % class_to_name[curcls]] = mAPbbox_R40[j, 0, 0]
            ret_dict['%s_image/moderate_R40' % class_to_name[curcls]] = mAPbbox_R40[j, 1, 0]
            ret_dict['%s_image/hard_R40' % class_to_name[curcls]] = mAPbbox_R40[j, 2, 0]

In [34]:
print(result) # 模板+内容

# print(ret_dict) #内容
# print("=======================")
# '''  
# {'Car_3d/easy_R40': 0.0, 'Car_3d/moderate_R40': 0.0, 'Car_3d/hard_R40': 0.0, 'Car_bev/easy_R40': 0.0, 'Car_bev/moderate_R40': 0.0, 'Car_bev/hard_R40': 0.0, 'Car_image/easy_R40': 0.0, 'Car_image/moderate_R40': 0.0, 'Car_image/hard_R40': 0.0, 
# 'Pedestrian_3d/easy_R40': 0.0, 'Pedestrian_3d/moderate_R40': 0.0, 'Pedestrian_3d/hard_R40': 0.0, 'Pedestrian_bev/easy_R40': 0.0, 'Pedestrian_bev/moderate_R40': 0.0, 'Pedestrian_bev/hard_R40': 0.0, 'Pedestrian_image/easy_R40': 0.0, 'Pedestrian_image/moderate_R40': 0.0, 'Pedestrian_image/hard_R40': 0.0, 
# 'Cyclist_3d/easy_R40': 0.0, 'Cyclist_3d/moderate_R40': 0.0, 'Cyclist_3d/hard_R40': 0.0, 'Cyclist_bev/easy_R40': 0.0, 'Cyclist_bev/moderate_R40': 0.0, 'Cyclist_bev/hard_R40': 0.0, 'Cyclist_image/easy_R40': 0.0, 'Cyclist_image/moderate_R40': 0.0, 'Cyclist_image/hard_R40': 0.0}
# '''
# # 字典输出处理
# for key in ret_dict.keys():
#     print("%s: %s"%(key, str(ret_dict[key])))


```
print(result) 
Car AP@0.70, 0.70, 0.70:
Car AP@0.70, 0.70, 0.70:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Car AP_R40@0.70, 0.70, 0.70:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Car AP@0.70, 0.50, 0.50:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Car AP_R40@0.70, 0.50, 0.50:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Pedestrian AP@0.50, 0.50, 0.50:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Pedestrian AP_R40@0.50, 0.50, 0.50:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Pedestrian AP@0.50, 0.25, 0.25:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Pedestrian AP_R40@0.50, 0.25, 0.25:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Cyclist AP@0.50, 0.50, 0.50:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Cyclist AP_R40@0.50, 0.50, 0.50:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Cyclist AP@0.50, 0.25, 0.25:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
Cyclist AP_R40@0.50, 0.25, 0.25:
bbox AP:0.0000, 0.0000, 0.0000
bev  AP:0.0000, 0.0000, 0.0000
3d   AP:0.0000, 0.0000, 0.0000
```